In [38]:
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

api = wandb.Api(timeout=60)

In [75]:
# shape 4,8,8,2, extension from 0 to 20 levels
sweep_ids=['wpcowdl5', 'j9lvvxjg', 'a5kr3muw', 'iat6x9bh']
untapped_potential = 'untapped-potential'
_active_weights_abs = "active-weights-abs"
_pparams = "pparams"
_untapped_potential = 'untapped-potential'

# with shape 4,4,4,4,2
if True:
    #sweep_ids=['t1mp5nhi', 'c3gin27l', '82wq0l59', 'vk3ftk2q','vpqib95x'] # 3 hidden layers
    sweep_ids=['f9zju9yb', '01ayiisq']  # one hidden layer
    _pparams = "meta/" + _pparams
    _untapped_potential = 'graph/' + _untapped_potential
    _active_weights_abs = "graph/" + _active_weights_abs

runs = []
for sweep_id in sweep_ids:
    runs += [run for run in api.sweep('concat_moons/' + sweep_id).runs]

#histories = [run.history() for run in runs]
configs = [run.config for run in runs]

In [76]:
# single layer. network has 4, 20, 2 base shape
t1 = [26550,18054,12276,8346,5676,3858,2622,1782,1212,828,564,384,258,174,120]

# two layer. network has 4, 8, 8, 2 base shape
t2 = [170560,115591,78952,53815,36472,24640,16891,11440,7735,5320,3591,2491,1672,1147,775,520,352,247,160,112]

# three layer. 4, 4, 4, 4, 2 base shape
t3 = [26676,18236,12320,8316,5720,3956,2660,1856,1196,836,540,360,260,176,108,80,56]

pd.DataFrame.from_dict({'t1': reversed(t1), 't2': reversed(t2), 't3':reversed(t3)},orient='index').T

,t1,t2,t3
0,120.0,112.0,56.0
1,174.0,160.0,80.0
2,258.0,247.0,108.0
3,384.0,352.0,176.0
4,564.0,520.0,260.0
5,828.0,775.0,360.0
6,1212.0,1147.0,540.0
7,1782.0,1672.0,836.0
8,2622.0,2491.0,1196.0
9,3858.0,3591.0,1856.0


In [77]:
shapes, level, pparams = [], [], []
for c in configs:
    shapes.append(c['model_shape'][1])
    level.append(c['extension_levels'])
    pparams.append(c['param_trajectory'][0])
pd.DataFrame({
    'shape' : shapes,
    'level': level,
    'pparams':pparams
}).sort_values('level').drop_duplicates().reset_index(drop=True).drop('level', axis=1).to_csv('1layer.csv')

In [113]:
l1 = pd.read_csv('1layer.csv').rename(columns={'shape':'layer-1-shape', 'pparams': 'layer-1-prunable-parameters'})
l2 = pd.read_csv('2layer.csv').rename(columns={'shape':'layer-2-shape', 'pparams': 'layer-2-prunable-parameters'})
l3 = pd.read_csv('3layer.csv').rename(columns={'shape':'layer-3-shape', 'pparams': 'layer-3-prunable-parameters'})

In [115]:
pd.merge(l1, 
    pd.merge(l2, l3, on='Unnamed: 0', how='outer'),
    on='Unnamed: 0',
    how='outer'
).drop('Unnamed: 0', axis=1).astype('Int64').to_csv('table.csv')


In [84]:
pd.read_csv('1layer.csv').keys()

Index(['Unnamed: 0', 'shape', 'pparams'], dtype='object')